In [1]:
from bs4 import BeautifulSoup
from posl import load_posl_df
from glob import glob
from datetime import datetime
import os
import pandas as pd
import re
%matplotlib tk

In [2]:
def is_comment(tag):
    text = tag.getText()
    return text.startswith("(") and text.endswith(")")

def is_speaker(tag):
    return not is_comment(tag) and not tag['who'].lower().startswith(('#komentarz', '#glos', '#marszalek', '#poslowie', '#wicemarszalek', '#gwar')) 

def get_speaker_name(speaker_dict, tag):
    speaker_id = tag.find(is_speaker)['who'][1:]
    [name, surname] = speaker_dict[speaker_id].split(' ')[-2:]
    return f'{name} {surname}'

def get_id(doc_id, tag):
    return f'{doc_id}-{tag["xml:id"]}'

def parse_folder(folder):
    with open(os.path.join(folder, "header.xml"), 'r') as header_file:
        header = BeautifulSoup(header_file, 'lxml')

    doc_id =  header.find("teiheader")["xml:id"]
    doc_date = datetime.strptime(header.find("date").text, '%Y-%m-%d')
    doc_title = header.find("title").text
    doc_speakers = {x["xml:id"]: x.text.strip() for x in header.find_all("person")}

    with open(os.path.join(folder, "text_structure.xml"), 'r') as data_file:
        data = BeautifulSoup(data_file, 'lxml')
    
    divs = data.find_all('div')
    texts = [[get_id(doc_id, div), get_speaker_name(doc_speakers, div), doc_date, ' '.join([x.text for x in div.find_all(is_speaker)])] for div in divs if div.find(is_speaker)]
    texts = [t for t in texts if len(t[3]) >= 200]
    df = pd.DataFrame(texts, columns=["id", "speaker", "date", "text"])

    return df   

In [3]:
pp_all = 'corpus/2015-2019/sejm/posiedzenia/pp/*'

frames = [parse_folder(f) for f in glob(pp_all)]

corpus = pd.concat(frames)

In [4]:
posl = load_posl_df()
with open('posl/kadencje_start') as f:
    all_kadencje = sorted([datetime.strptime(x.strip(), '%Y-%m-%d') for x in f], reverse=True)

posel = []
klub = []
lista = []
okreg = []

for _, row in corpus.iterrows():
    speaker = row.speaker
    kadencja = next((k for k in all_kadencje if row.date >= k))

    found = posl[(posl.rok == kadencja.year) & (posl.posel.str.endswith(speaker))]

    if len(found == 1):
        found = found.squeeze()

        posel.append(found.posel)
        lista.append(found.partia or found.lista or found.klub or '')
        klub.append(found.klub or '')
        okreg.append(found.okreg)
    else:
        posel.append('')
        lista.append('')
        klub.append('')
        okreg.append('')

corpus['posel'] = posel
corpus['klub'] = klub
corpus['lista'] = lista
corpus['okreg'] = okreg

corpus = corpus[corpus.posel != '']

In [8]:
corpus.to_csv('parsed/corpus/2015.csv')

In [7]:
posl = load_posl_df()
corpus = pd.read_csv('parsed/corpus/2015.csv', index_col=0, na_filter=False)

In [7]:
corpus.lista.value_counts().plot(kind='barh')

<AxesSubplot:>

In [57]:
corpus['text_lengts'] = [len(x) for x in corpus['text']]

In [58]:
corpus['text_lengts'].hist(bins=500)

<AxesSubplot:>

In [59]:
corpus.sort_values(by=['text_lengts'],ascending=False)

,id,speaker,date,text,posel,klub,okreg,lista,text_lengts
5,PPC-201519-sjm-ppxxx-00048-01-div-12,Mirosław Maliszewski,2017-09-27,Panie Marszałku! Pani Premier! Panie i Panowie...,Mirosław Maliszewski,Klub Parlamentarny Polskie Stronnictwo Ludowe ...,Radom,Polskie Stronnictwo Ludowe,90450
99,PPC-201519-sjm-ppxxx-00020-02-div-202,Stanisław Piotrowicz,2016-06-09,Pani Marszałek! Wysoka Izbo! Zanim przystąpię ...,Stanisław Piotrowicz,Klub Parlamentarny Prawo i Sprawiedliwość,Krosno,Prawo i Sprawiedliwość,74513
1,PPC-201519-sjm-ppxxx-00035-02-div-4,Witold Waszczykowski,2017-02-09,Panie Prezydencie! Panie Marszałku! Szanowna P...,Witold Waszczykowski,Klub Parlamentarny Prawo i Sprawiedliwość,Sieradz,Prawo i Sprawiedliwość,58792
1,PPC-201519-sjm-ppxxx-00010-02-div-4,Witold Waszczykowski,2016-01-29,Panie Prezydencie! Panie Marszałku! Pani Premi...,Witold Waszczykowski,Klub Parlamentarny Prawo i Sprawiedliwość,Sieradz,Prawo i Sprawiedliwość,57856
1,PPC-201519-sjm-ppxxx-00039-03-div-4,Grzegorz Schetyna,2017-04-07,"Panie Marszałku! Ten się śmieje, kto się śmiej...",Grzegorz Schetyna,Klub Parlamentarny Platforma Obywatelska - Koa...,Kielce,Platforma Obywatelska,52446
...,...,...,...,...,...,...,...,...,...
234,PPC-201519-sjm-ppxxx-00039-02-div-475,Szymon Giżyński,2017-04-06,Panie Marszałku! Wysoki Sejmie! Nowoczesna i P...,Szymon Giżyński,Klub Parlamentarny Prawo i Sprawiedliwość,Częstochowa,Prawo i Sprawiedliwość,201
309,PPC-201519-sjm-ppxxx-00081-02-div-625,Bożenna Bukiewicz,2019-05-16,"Oświadczam, że w dniu 16 maja 2019 r. w głosow...",Bożenna Bukiewicz,Klub Parlamentarny Platforma Obywatelska - Koa...,Zielona Góra,Platforma Obywatelska,201
230,PPC-201519-sjm-ppxxx-00015-02-div-466,Sławomir Neumann,2016-03-31,"Panie marszałku, chciałbym tutaj dla higieny p...",Sławomir Neumann,Klub Parlamentarny Platforma Obywatelska - Koa...,Gdańsk,Platforma Obywatelska,200
31,PPC-201519-sjm-ppxxx-00076-02-div-66,Elżbieta Duda,2019-01-17,Drugie pytanie uzupełniające. Czy na podstawie...,Elżbieta Duda,Klub Parlamentarny Prawo i Sprawiedliwość,Kraków,Prawo i Sprawiedliwość,200
